First cut at Full Vehicle Dynamics Model for Performance Trends using equations and values from Form1.vb and FrontSuspension.m. 

Here are the initial assumptions:

Use static values for LF, RF trackwidth
LR & RR trackwidth is the same as LF & RF
Use static panhard angle
Use previous regression equation for LFIC, LFAD, LFFVSA, RFIC, RFAD, RFFVSA, LRAS, RRAS based on Pitch and Roll


Vehicle Suspension Parameters

In [2]:
CGy = 5.1 #Center of Gravity y, in
CGz = 16 #Center of Gravity z, in
LFTW = 32.29 #LF Trackwidth, in
RFTW = 32.25 #RF Trackwidth, in
LRTW = LFTW #LR Trackwidth, in 
RRTW = RFTW #RR Trackwidth, in
WB = 108 #Wheelbase, in
P2CL = 20 #Panhard to Centerline, in
LA2CL = 24 #Lower Arm to Centerline, in
P2LR = LRTW - P2CL #Panhard to LR, in
LRP = 18 #LR Panhard, in
RWR = 13.5 #Rear Wheel Radius, in
PAng = -3.3/57.3 #Panhard angle, converted from deg to rad


Vehicle Weight Parameters

In [4]:
Nose = 49 #Nose Weight Pct, %
Cross = 51 #Cross Weight Pct, %
WLeft = 48 #Left Side Weight Pct, %
WRight = (100-WLeft) #Right Side Weight Pct, %
FBrakeBias = 50 #Front Brake Bias Pct, %
VehWt = 2850 #Vehicle Weight, lbs
VehM = VehWt/32.2/12 #Convert to Mass
USLFWt = 170 #Left Front Unsprung Weight, lbs
USLFM = USLFWt/32.2/12 #Left Front Unsprung Mass 
USRFWt = 170 #Right Front Unsprung Weight, lbs
USRFM = USRFWt/32.2/12 #Right Front Unsprung Mass 
USRWt = 429 #Left Rear Unsprung Weight, lbs
USRM = USRWt/32.2/12 #Rear Unsprung Mass 
LFStatic = 700 #Left Front Scale Weight, lbs
RFStatic = 700 #Right Front Scale Weight, lbs
LRStatic = 868 #Left Rear Scale Weight, lbs
RRStatic = 528 #Right Rear Scale Weight, lbs
DragCoef = .42 #Vehicle Drag Coef
FrontArea = 21 #Vehicle Frontal Area, ft^2
a = (Nose/100)*WB #Long dist from front axle to CG
b = WB-a #Long dist from rear axle to CG

Suspension Rates

In [ ]:
# Tire Spring Rate
KtLF = 1500 #Left Front Tire Spring Rate, lb/in
KtRF = 3800 #Right Front Tire Spring Rate, lb/in
KtLR = 1800 #Left Rear Tire Spring Rate, lb/in
KtRR = 2500 #Right Rear Tire Spring Rate, lb/in

# Wheel Rate
KwLF = 200 #Left Front Wheel Rate, lb/in
KwRF = 700 #Right Front Wheel Rate, lb/in
KwLR = 125 #Left Rear Wheel Rate, lb/in
KwRR = 475 #Right Rear Wheel Rate, lb/in

# Shock Rate
KshLF = 100 #Left Front Shock Rate, lb/in
KshRF = 200 #Right Front Shock Rate, lb/in
KshLR = 75 #Left Rear Shock Rate, lb/in
KshRR = 250 #Right Rear Shock Rate, lb/in

# Roll Bar Rate
Kfarb = 250000 #Front Rollbar Rate, lb/in
Krarb = 80000 #Rear Rollbar Rate, lb/in

Circle Track Analyzer Outputs
Dive, in
Roll, deg
VehLFWt, lbs
VehRFWt, lbs
VehLRWt, lbs
VehRRWt, lbs
RPM
Torque (missing?)
LongAccel, g's
LatAccel, g's
Downforce, lbs
Speed, mph

Initial test with 5 timesteps to sort code. Should be able to read this file in.

In [6]:
Dive = [0.55, -0.21, -0.33, -0.42, -0.49] 
Roll = [0.42, 1.44, 1.4, 1.31, 1.23]
VehLFWt =[355, 369, 395, 418, 455]
VehRFWt = [986, 920, 851, 798, 764]
VehLRWt = [856, 925, 991, 1038, 1053]
VehRRWt = [906, 833, 751, 686, 634]
RPM = [4316, 4509, 4879, 5251, 5616]
LongAccel = [0, 0.52, 0.53, 0.52, 0.51]
LatAccel = [0.89, 0.88, 0.71, 0.52, 0.37]
Downforce = [317, 310, 246, 172, 115]
Speed = [67, 70, 76, 82, 88]

Perform preliminary calculations that will be used later based on Circle Track Analyzer Output
VertAccel, g's
Drag, lbs
LongForce, lbs
LatForce, lbs

In [22]:
# Vertical Acceleration

VertAccel = [Downforce[i]/VehWt for i in range(len(Downforce))]
#print(VertAccel)

# Convert Speed to ft/s
Speed_fps = [Speed[i]/(5280/3600) for i in range(len(Speed))]
#print(Speed_fps)

# Drag
Drag = [DragCoef*FrontArea*0.00258* Speed_fps[i]**2 for i in range(len(Speed_fps))]
print(Drag)

# Longitudinal Force
Fx = [(LongAccel[i]*VehWt)+Drag[i] for i in range(len(LongAccel))]
#print(Fx)

# Lateral Force
Fy = [(LatAccel[i]*VehWt) for i in range(len(LatAccel))]
#print(Fy)

[47.48703489669423, 51.83481198347109, 61.10160694214876, 71.13005628099175, 81.92016000000002]


Calculate left front and right front instant center, anti dive and front view swing angle using previous regression coefficients. Moving forward, this should be regressed on the fly using dive and roll, or interpolated using dive.

Calculate left rear and right rear anti squat percentage. Moving forward this should be regressed on the fly using the dive and roll, or interpolated using dive.

In [26]:
# Left front instant center
LFIC = [6.3733 + -1.54 * Dive[i] + 0.3317 * Roll[i] + 0.1 * Dive[i]**2 + 0.015 * Roll[i]**2 + -0.08 * Dive[i] * Roll[i]
       for i in range(len(Dive))]
#print(LFIC)

# Left front anti dive
LFAD = [3.7976 + -0.0116 * Dive[i] + 0.1149 * Roll[i] + -0.042 * Dive[i]**2 + -0.0135 * Roll[i]**2 + 0.0613 * Dive[i] * Roll[i]
       for i in range(len(Dive))]
#print(LFAD)

# Left front front view swing angle
LFFVSA = [31.0644 + -1.521 * Dive[i] + 1.1083 * Roll[i] + 0.0802 * Dive[i]**2 + 0.0222 * Roll[i]**2 + -0.1025 * Dive[i] * Roll[i]
       for i in range(len(Dive))]
#print(LFFVSA)

# Right front instant center
RFIC = [6.7767 + -1.625 * Dive[i] + -0.3067 * Roll[i] + 0.115 * Dive[i]**2 + 0.01 * Roll[i]**2 + 0.08 * Dive[i] * Roll[i]
       for i in range(len(Dive))]
#print(RFIC)

# Right front anti dive
RFAD = [-2.4693 + -0.6151 * Dive[i] + -0.2664 * Roll[i] + -0.0652 * Dive[i]**2 + -0.0032 * Roll[i]**2 + -0.0297 * Dive[i] * Roll[i]
       for i in range(len(Dive))]
#print(RFAD)

# Right front front view swing angle
RFFVSA = [34.5054 + -2.2535 * Dive[i] + -1.4583 * Roll[i] + 0.1298 * Dive[i]**2 + 0.0363 * Roll[i]**2 + 0.158 * Dive[i] * Roll[i]
       for i in range(len(Dive))]
#print(RFFVSA)

# Left rear anti squat percentage
LRAS = [28.2053 + -19.5437 * Dive[i] + 12.8093 * Roll[i] + -1.2993 * Dive[i]**2 + 0.0467 * Roll[i]**2 + 0.9687 * Dive[i] * Roll[i]
       for i in range(len(Dive))]
#print(LRAS)

# Right rear anti squat percentage
RRAS = [66.4164 + -16.8635 * Dive[i] + -12.7975 * Roll[i] + -1.2193 * Dive[i]**2 + 0.0487 * Roll[i]**2 + -0.9245 * Dive[i] * Roll[i]
       for i in range(len(Dive))]
#print(RRAS)

[22.675140329999998, 50.50147211, 52.09023983000001, 52.51180361, 52.71200701000001]
[51.19271792999999, 51.856116990000004, 54.45464422999999, 57.11149444999999, 59.276710449999996]


Determine how to split longitudinal force between individual wheels during braking and acceleration

In [35]:
# Front Longitudinal Ratio

LFRatio = (LFTW + CGy)/(LFTW+RFTW)
RFRatio = (RFTW - CGy)/(LFTW+RFTW)
#print(LFRatio, RFRatio)

# Rear Longitudinal Ratio

LRRatio = (LRTW + CGy)/(LRTW+RRTW)
RRRatio = (RRTW - CGy)/(LRTW+RRTW)
#print(LRRatio, RRRatio)

# Seperate longitudinal forces based on braking or acceleration
# can loop, but trying to use list comprehension for better programing
Drive = [y for y in LongAccel if y > 0 for num in Fx]
print(Drive)

[0.52, 0.52, 0.52, 0.52, 0.52, 0.53, 0.53, 0.53, 0.53, 0.53, 0.52, 0.52, 0.52, 0.52, 0.52, 0.51, 0.51, 0.51, 0.51, 0.51]
